<div class="alert alert-success">
    <h1 align="center">Assignment 2: Logistic Regression</h1>
    <h3 align="center">Machine Learning Course (Fall 2018)</h3>
    <h5 align="center">Seyed Naser RAZAVI <a href="https://www.snrazavi.ir/ml-2018/">ML2018</a></h5>
</div>

Note that the contents in this assigment and the following assignments are heavily inspired from the amazing machine learning course presented at Stanford by Andrew Ng.

However, here in this series assignments for our machine learning course, we have used **Python** language which we think is much more suitable than Matlab for artificial intelligence students. Also we have changed and modified some parts of the assignments and the related descriptions for those parts.

In this assignment you only need to implement the following functions:
- `sigmoid(x)`: 
 - this is a warmup exercize and not graded.
- `logistic_regression_cost(theta, X, y)`: 
 - cost function for logistic regression without regularization.
- `grads(theta, X, y)`: 
 - the gradients of the above function.
- `predict(theta, X):`: 
 - prediction function which predicts the label or class of the input data.
- `regularized_logistic_regression_cost(theta, X, y, reg)`: 
 - cost function for logistic regression with regularization.
- `regularized_grads(theta, X, y, reg)`: 
 - the gradients of the above function.
    

## Logistic Regression

First, let us import two useful python library which we will use through this assinment.
- The first library is `numpy` to work with N-dimensional arrays.
- The second library is `matplotlib` for plotting.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from utils import *

# setup for printing numpy arrays
np.set_printoptions(precision=2)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 120

## Part 1: Logistic Regression

In the first part of this exercise, we'll build a logistic regression model to predict whether a student gets admitted to a university.

Suppose that you are the administrator of a university department and you want to determine each applicant's chance of admission based on their results on two exams. You have historical data from previous applicants that you can use as a training set for logistic regression. For each training example, you have the applicant's scores on two exams and the admissions decision. 

### Data

The data that you need for this part of the assignment is given to you in the text file `data/ex2data1.txt`. Here, you can see the first 10 rows from this file. The first two columns or the **input features** are the applicant's scores on two different exams and the third column (or the **target output**) is the admissions decesion. Here, `0` means "Not Admitted" and `1` means "Admitted". 

```python
34.62365962451697,78.02469281536240,0
30.28671076822607,43.89499752400101,0
35.84740876993872,72.90219802708364,0
60.18259938620976,86.30855209546826,1
79.03273605071010,75.34437643691030,1
45.08327747668339,56.31637178153050,0
61.10666453684766,96.51142588489624,1
75.02474556738889,46.55401354116538,1
76.09878670226257,87.42056971926803,1
84.43281996120035,43.53339331072109,1
```

Now, our goal is to build a classification model that estimates the probability of admission based on the exam scores.

### Loading data

The function `load_data()` has already been set up to load this data for you. It recieves the file name which contains the data (in this case `data/ex2data1.txt`) and returns `X` and `y`. This function is implemented for you in the file `utils.py` and it is imported at the begining of this notebook using the following `import` statement.
```python
    from utils import *
```
 Please see the implementation and make sure that you understand every detail.

In [ ]:
X, y = load_data(fname='data/ex2data1.txt', include_bias=True)

print(X.shape)
print(y.shape)

Please note that unlike the previous assignment, here in this assignment `y` is a 1d vector.

### Visualizing data

Before starting on any task, it is often useful to understand the data by visualizing it. For this dataset, you can use a scatter plot to visualize the data, since it has only two properties to plot (`profit` and `population`). (Many other problems that you will encounter in real life are multi-dimensional and can’t be plotted on a 2-d plot.) 

Now, you can call `plot_data` to create a scatter plot of the data. Again, we have implemented this function for you and imported it at the begining of this notebook. Run the following cell to see the data.

In [ ]:
plot_data(X, y, labels=['Not Admitted', 'Admitted'])

### Warmup exercize: sigmoid function 

Before you start with the actual cost function, recall that the logistic regression hypothesis is deﬁned as:
$$h_\theta(x)=g(\theta^T x)$$

where function g is the sigmoid function. The sigmoid function is deﬁned as:
$$g(z)=\frac{1}{1 + e^{-z}}$$

Your first step is to implement this function so it can be called by the rest of your program. When you are finished, try testing a few values by calling `sigmoid(x)`. For large positive values of `x`, the sigmoid should be close to `1`, while for large negative values, the sigmoid should be close to `0`. Evaluating `sigmoid(0)` should give you exactly `0.5`. 

Note that your code should also work with vectors and matrices. For a matrix, your function should perform the sigmoid function on every element.

In [ ]:
def sigmoid(x):
    """ Sigmoid function.
    
    Parameters:
    -----------
        - x: input, can be a scaler value or a numpy array
        
    Returns:
    --------
        - The sogmoid of x. 
        
    Examples:
    --------
        - sigmoid(0)    -> 0.5
        - sigmoid(100)  -> 1.0
        - sigmoid(-100) -> 0.0
    """
    raise NotImplementedError("You need to implement `sigmoid` function.")

Now, you can run the following cells to see if your implementation of sigmoid function is correct.

In [ ]:
# testing sigmoid for scaler values
print("sigmoid(   0) = %.2f" % sigmoid(0))
print("sigmoid( 100) = %.2f" % sigmoid(100))
print("sigmoid(-100) = %.2f" % sigmoid(-100))

Let's do a quick sanity check to make sure the implementation of sigmoid function is correct.

In [ ]:
x = np.arange(-10, 10, 0.01)
plt.plot(x, sigmoid(x), 'r')
plt.show()

### Cost function and gradient

Now you will implement the cost function and gradient for logistic regression. Complete the code in `logistic_regression_cost()` to return the cost and the `grads()` to return the gradient of the cost function.

Recall that the cost function in logistic regression is

$$J(\theta) = \frac{1}{m} \sum_{i=1}^{m} [-y^{(i)} \log(h_\theta(x^{(i)})) - (1 - y^{(i)}) \log(1 - h_\theta(x^{(i)}))]$$

and the gradient of the cost is a vector θ where the jth element (for j = 0,1,...,n) is defined as follows:


$$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})x^{(i)}_{j}$$

Note that while this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $h_\theta(x)$.

In [ ]:
def logistic_regression_cost(theta, X, y):
    """ Vectorized implementation of logistic regression cost function.
    
    Parameters:
        - theta: Parameters, a 1d array of shape (n + 1,)
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 1d array of shape (m,)
        
    Returns:
        - logistic regression cost
    """
    raise NotImplementedError("You need to implement `logistic_regression_cost` function.")

def grads(theta, X, y):
    """ Vectorized implementation of the gradient of logistic regression cost function.
    
    Parameters:
        - theta: Parameters, a 1d array of shape (n + 1,)
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m,)
        
    Returns:
        - gradient of the logistic regression cost function, a 2d array of shape (n + 1,)
    """
    raise NotImplementedError("You need to implement `grads` function.")

To check your implementation of cost function, run the following cell. If you have implemented the cost function correctly, you should see that the cost is about `0.693` and gradient vector is about `[-0.1, -12.01, -11.26]`.

In [ ]:
# testing cost function and gradient
theta = np.zeros((3,))
print("cost = %.3f" % logistic_regression_cost(theta, X, y))
print("gradient = %s" % grads(theta, X, y).ravel())

### Learning parameters using `scipy.optimize.minimize`

In the previous assignment, you found the optimal parameters of a linear regression model by implementing gradent descent. You wrote a cost function and calculated its gradient, then took a gradient descent step accordingly. This time, instead of taking gradient descent steps, you will use a function in `scipy.optimize` libray called `minimize`.

Let's us first import this function to use it later.

In [ ]:
from scipy.optimize import minimize

The `scipy.optimize.minimize` is an optimization solver that finds the minimum of a function. For logistic regression, you want to optimize the cost function `J(θ)` with parameters `θ`.

Concretely, you are going to use `minimize` to find the best parameters `θ` for the logistic regression cost function, given a fixed dataset (of `X` and `y` values). You will pass to `minimize` the following inputs:
- **Initial values**: The initial values of the parameters we are trying to optimize.
- **Cost function**: A function that, when given the training set and a particular `θ`, computes the logistic regression cost for the dataset (`X`, `y`).
- **Gradients**: A function, when given the training set and a particular `θ`, computes gradient with respect to `θ` for the dataset (`X`, `y`).


```python
minimize(cost_function, x0=theta0, args=(X, y), method='CG', jac=grads)
```

In [ ]:
result = minimize(logistic_regression_cost, x0=np.zeros((3,)), args=(X, y), method='bfgs', jac=grads)

In [ ]:
result

In [ ]:
theta_opt = result.x  # optimal theta
print(theta_opt)

Now, we will call your cost function using the optimal parameters of θ. You should see that the cost is about `0.203`. 

In [ ]:
cost = logistic_regression_cost(theta_opt, X, y)
print("cost = {:.3f}".format(cost))

This final `θ` value will then be used to plot the decision boundary on the training data, resulting in a figure similar to Figure 2. We also encourage you to look at the code in `plot_decision_boundary()` to see how to plot such a boundary using the `θ` values. This function is implemented for you in the file `utils.py`.

If everything is ok, you should see something like the following:

<img src='imgs/part_1_log_reg_decision_boundary.png' width='75%'>

In [ ]:
plot_decision_boundary(theta_opt, X, y)

### Evaluating the classifier

After learning the parameters, you can use the model to predict whether a particular student will be admitted. For a student with an Exam 1 score of `45` and an Exam 2 score of `85`, you should expect to see an admission probability of `0.776`. 

Another way to evaluate the quality of the parameters we have found is to see how well the learned model predicts on our training set. In this part, your task is to complete the code in `predict()`. The predict function will produce `1` or `0` predictions given a dataset and a learned parameter vector `θ`. 

In [ ]:
def predict(theta, X):
    raise NotImplementedError("You need to implement `predict` function.")

In [ ]:
x_test = np.array([[1, 45, 85]])
print(sigmoid(x_test @ theta_opt.T))
print(predict(theta_opt, x_test))

## Part 2: Regularized Logistic Regression

In this part of the exercise, you will implement regularized logistic regression to predict whether microchips from a fabrication plant passes quality assurance (QA). During QA, each microchip goes through various tests to ensure it is functioning correctly. 

Suppose you are the product manager of the factory and you have the test results for some microchips on two diﬀerent tests. From these two tests, you would like to determine whether the microchips should be accepted or rejected. To help you make the decision, you have a dataset of test results on past microchips, from which you can build a logistic regression model.

### Visualizing data

Similar to the previous part of this exercise, `plot_data()` is used to generate a figure like Figure 3, where the axes are the two test scores, and the positive (y = 1, accepted) and negative (y = 0, rejected) examples are shown with different markers.


In [ ]:
X, y = load_data(fname='data/ex2data2.txt')

print(X.shape)
print(y.shape)

In [ ]:
plt.figure(figsize=(12, 8), dpi=120)
plot_data(X, y, xlabel='Microchip Test 1', ylabel='Microchip Test 2')

This data looks a bit more complicated than the previous example. In particular, you'll notice that there is no linear decision boundary that will perform well on this data. One way to deal with this using a linear technique like logistic regression is to construct features that are derived from polynomials of the original features. 

Let's start by creating a bunch of polynomial features using the `map_features()` function that we have implented for you in `utils.py`.

In [ ]:
degree = 8
X_poly = map_features(X[:, 1], X[:, 2], degree=degree)
print(X_poly.shape)

Now we need to modify the cost and gradient functions from part 1 to include the regularization term. Please complete the implementation of the two following function.

In [ ]:
def regularized_logistic_regression_cost(theta, X, y, reg):
    """ Vectorized implementation of logistic regression cost function.
    
    Parameters:
        - theta: Parameters, a 1d array of shape (n + 1,)
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 1d array of shape (m,)
        
    Returns:
        - logistic regression cost
    """
    raise NotImplementedError("You need to implement `regularized_logistic_regression_cost` function.")


def regularized_grads(theta, X, y, reg):
    """ Vectorized implementation of the gradient of logistic regression cost function.
    
    Parameters:
        - theta: Parameters, a 1d array of shape (n + 1,)
        - X: Inputs, a 2d array of shape (m, n + 1)
        - y: Target values, a 2d array of shape (m,)
        
    Returns:
        - gradient of the logistic regression cost function, a 2d array of shape (n + 1,)
    """
    raise NotImplementedError("You need to implement `regularized_grads` function.")

If you have implented these two function correctly, you should see that for $\theta$ equals to zero, the cost is about `0.693`.

In [ ]:
theta = np.zeros(X_poly.shape[1])
cost = regularized_logistic_regression_cost(theta, X_poly, y, 1.0)
print('cost = {:.3f}'.format(cost))

Now we can use the same optimization function from part 1 to compute the optimal solution.

In [ ]:
reg = 0.0
result = minimize(
    regularized_logistic_regression_cost, 
    x0=theta, 
    args=(X_poly, y.ravel(), reg), 
    method='bfgs', 
    jac=regularized_grads)

In [ ]:
result.x

Finally, we can use the prediction function from part 1 to see how accurate our solution is on the training data. After running the following cell, you should see that the accuracy is about `88.14%`.

In [ ]:
def accuracy(y_pred, y_true):
    return 100. * np.sum(y_pred == y_true) / len(y_true)

In [ ]:
theta_opt = result.x
predictions = predict(theta_opt, X_poly)
acc = accuracy(predictions, y)
print('accuracy = {:.2f}%'.format(acc))

Run the following cell to see the classification result. If everything is Ok, you should see something like this:

<img src='imgs/part_2_lambda_0.png' width='75%'>

With a small λ, you should find that the classiﬁer gets almost every training example correct, but draws a very complicated boundary, thus **overfitting** the data.

In [ ]:
title = "Degree = {}, $\lambda$ = {}, accuracy = {:.2f}".format(degree, reg, acc)
plot_decision_boundary(theta_opt, X_poly, y.ravel(), 
                       xlabel='Microchip Test 1', ylabel='Microchip Test 2', 
                       degree=degree, title=title)

Now by trying different value for `degree` and `reg` (regularization strength), find a an accurate and smooth decision boundary for the given data.

In [ ]:
# change the value of these two hyper-parameters
degree = 8
reg = 100.0

# create polynomial features
X_poly = map_features(X[:, 1], X[:, 2], degree=degree)

# initialize theta to 0
theta = np.zeros(X_poly.shape[1])

# optimize cost function to find the optimal parameters
result = minimize(
    regularized_logistic_regression_cost, 
    x0=theta, 
    args=(X_poly, y.ravel(), reg), 
    method='bfgs', 
    jac=regularized_grads)

# compute accuracy
theta_opt = result.x
predictions = predict(theta_opt, X_poly)
acc = accuracy(predictions, y)

# plot decision boundary
title = "Degree = {}, $\lambda$ = {}, accuracy = {:.2f}".format(degree, reg, acc)
plot_decision_boundary(theta_opt, X_poly, y.ravel(), 
                       xlabel='Microchip Test 1', ylabel='Microchip Test 2',
                       degree=degree, title=title)

## Using Scikit-learn library

Although we implemented these algorithms from scratch, it's worth noting that we could also use a high-level python library like scikit-learn to solve this problem.

In [ ]:
from sklearn import linear_model
C = 1e10
model = linear_model.LogisticRegression(penalty='l2', C=C, fit_intercept=False)
model.fit(X_poly, y.ravel());

In [ ]:
acc = model.score(X_poly, y) * 100

theta_opt = np.array(model.coef_)

title = "Degree = {}, C = {}, accuracy = {:.2f}".format(degree, C, acc)
plot_decision_boundary(theta_opt, X_poly, y.ravel(), 
                       xlabel='Microchip Test 1', ylabel='Microchip Test 2',
                       degree=degree, title=title)

## Submittion

After finishing the assignment, please send **only** this jupyter notebook file via Piazza as a **private** post before the due date. Before submitting your file, be sure to read the following terms and conditions.

**Due date**: November, 9, 2018.

### Terms and Conditions
Please consider the following terms and conditions and take them as serious as you can:
- If you send your code as a public code in Piazza, you will fail in the course.
- If you share all or some parts of your results on the internet or with your friends and classmates in any form, you will fail the course.
- If you submit codes that is not your work, (for example by copying your friend's codes), you will fail the course.
- Any other attempt to cheat, will result in a failure in the course.
- By submmiting this file to Piazza, you confirm that you have read and accepted the above terms and conditions.